In [15]:
import OffsetArrays
using Plots
using Revise
using ProgressMeter
using SparseArrays, LinearAlgebra, IndexedGraphs, Graphs
using IterTools
using DelimitedFiles
include("../src/functions.jl") #pop dynamics functions
include("../src/observables.jl") #utils functions for calculating observables and saving them

inf_vs_dil_optimal (generic function with 1 method)

# Population dynamics for the messages

In [16]:
# S=0   I=1
# The times go from ti = 0 to ti=T+1
# when ti=0 it means that i is the zero patient
# when ti=T+1 it means that i is never infected

T = 8 # discrete time
N = 20000; #population size
γ = 0.03 # zero patient probability 
fr = 0.0 # noise of measure
grid = 21 # the grid of the phase diagram

dilRange = LinRange(0.0,1.0,grid) 
λRange = LinRange(0.01,0.8,grid) 

21-element LinRange{Float64, Int64}:
 0.01,0.0495,0.089,0.1285,0.168,0.2075,…,0.6025,0.642,0.6815,0.721,0.7605,0.8

In [ ]:
#here we choose what regime we want to study (λ VS dilution, λ VS γ, dilution VS fr ...)
degree_dist = Poisson(3)
inf_out = inf_vs_dil_mism(γ, λRange,0.3, N, T, degree_dist, fr , dilRange , tot_iterations = 100);

In [19]:
# Here we write the results in files.
samp = 4
graph = "er"

"er"

In [20]:
open("./dil_lam_$(graph)_$(samp)/ninf.txt","w") do io
    writedlm(io,inf_out[:,:,1]) 
end
for t = 1:T+1
    open("./dil_lam_$(graph)_$(samp)/AUC$(t).txt","w") do io
        writedlm(io,inf_out[:,:,t+1]) 
    end
    open("./dil_lam_$(graph)_$(samp)/Ov$(t).txt","w") do io
        writedlm(io,inf_out[:,:,t+1 + T + 1]) 
    end
    open("./dil_lam_$(graph)_$(samp)/L1_$(t).txt","w") do io
        writedlm(io,inf_out[:,:,t+1 + 2*T + 2]) 
    end
    open("./dil_lam_$(graph)_$(samp)/MSE$(t).txt","w") do io
        writedlm(io,inf_out[:,:,t+1 + 3*T + 3]) 
    end
end

In [18]:
inf_out

21×21×37 Array{Float64, 3}:
[:, :, 1] =
   5.0    5.0    5.0    5.0    5.0    5.0  …    4.0    5.0    5.0    4.0  3.0
   5.0    5.0    5.0    6.0    7.0    5.0       6.0    5.0    4.0    4.0  4.0
   7.0    7.0    5.0    6.0    7.0    5.0       6.0    6.0    5.0    6.0  4.0
   6.0    6.0    7.0    9.0    7.0    7.0       8.0    7.0    7.0    6.0  4.0
   7.0    7.0    8.0    7.0    8.0    8.0       9.0    8.0    8.0    7.0  5.0
   9.0    7.0    8.0    9.0   10.0   10.0  …    9.0   10.0    9.0    9.0  5.0
  12.0   12.0   10.0   12.0   13.0   10.0      10.0   11.0    9.0    8.0  5.0
  17.0   15.0   12.0   18.0   15.0   16.0      12.0   12.0   11.0    9.0  5.0
  15.0   24.0   18.0   18.0   20.0   15.0      15.0   16.0   12.0   10.0  5.0
  20.0   26.0   18.0   19.0   23.0   32.0      29.0   17.0   16.0   10.0  5.0
  34.0   26.0   29.0   28.0   37.0   34.0  …   29.0   19.0   15.0   11.0  5.0
  51.0   60.0   71.0   87.0  100.0   63.0     100.0   31.0   20.0   14.0  5.0
 100.0  100.0  100.0  10